In [3]:
import numpy as np
import math
# Boundary types:
# type 0: there is no sink
# type 1: only the corners are connected to a sink by a single edge
# type 2: all boundary vertices are connected to a sink through a single edge
# type 3: all noncorner boundary vertices are connected to a sink through a single edge,
#           and each corner is connected to the sink through two edges
# type 4: (for linear lattices) no global sink, yet each endpoint is connected to the other through a directed edge.
# type 5: (when lattice size is >= 2) each vertex on the far left is conected by a single edge to the corresponding vertex on the far right. 
#
# The third argument in each list is the number of edges if the vertex is on both the top/bottom and left/right,
# the second is the number of edges if the vertex is on either the top/bottom or the left/right but not both,
# and the first is the number of edges if the vertex is neither on the top/bottom or the left/right.
boundary_types = {
     0: [0, 0, 0],
     1: [0, 0, 1],
     2: [0, 1, 1],
     3: [0, 1, 2]
};

# If calling with boundary type 4, the graph must be linear!
def lattice_degree_matrix(m,n,boundarytype):
    deg_matrix = [];
    for i in range(0,m*n):
        row = [];
        for j in range(0,m*n):
            if (i == j):
                row.append(num_neighbors(i,m,n,boundarytype));
            else: row.append(0);
        deg_matrix.append(row);
    return np.array(deg_matrix);


# I number the vertices in the lattice from 0 to mn-1, increasing from left to right and 
# moving to the start of the next row as the end of a row is reached (the lattice has m
# rows and n columns).
def lattice_adjacency_matrix(m,n,boundarytype):
    vertices = np.reshape(np.arange(0,m*n),(m,n));
    adj_matrix = [];
    for i in range(0,m*n):
        row = [];
        for j in range(0,m*n):
            if (((i == j+1 or i == j-1) and max(i,j)%n != 0) or abs(i-j) == n): row.append(1);
            else: row.append(0);
        adj_matrix.append(row);

    if (boundarytype == 4):
        if (n == 1 and m == 1): adj_matrix[0][0]=1 #self loop in the case of a single vertex 
        elif (n == 1 and m != 1):
            adj_matrix[0][m-1] = 1
            adj_matrix[m-1][0] = 1
        elif (n != 1 and m == 1):
            adj_matrix[0][n-1] = 1
            adj_matrix[n-1][0] = 1

    if (boundarytype == 5):
        if (n == 1 and m == 1): adj_matrix[0][0]=1 #self loop in the case of a single vertex 
        else:
            for i in range(0, m):
                adj_matrix[n*i][n*i+n-1]=1
                adj_matrix[n*i+n-1][n*i]=1

    return np.array(adj_matrix);


# i is the vertex number (the vertices in the lattice are numbered from 0 to mn-1, increasing 
# from left to right and moving to the start of the next row as the end of a row is reached.
def num_neighbors(i,m,n,boundarytype):
        if (n==1 and m==1): return(1) #self loop in the case of a single vertex
        
        ontoporbottom = (0 <= i < n or 0 <= m*n-1 - i < n)
        onleftorright = (i%n == 0 or i%n == n-1);

        if boundarytype == 4:
            if (n!=1 and m!=1): return(-1)
            else: return(2)
            
        if boundarytype == 5:
            if (i == 0 or i == m*n-n or i == n-1 or i == m*n-1): return(3)
            elif (onleftorright): return(4)
            elif (ontoporbottom): return(3)
            else: return(4)

        edgestosink = boundary_types[boundarytype][ontoporbottom+onleftorright];

        #if boundarytype != 0 and (ontoporbottom or onleftorright):
        #    if (ontoporbottom and onleftorright): edgestosink = 1 if boundarytype==1 or boundarytype==2 else 2;
        #    else: edgestosink = 0 if boundarytype==1 else 1;

        if (n==1 or m==1):
            startingval = 3
        else:
            startingval = 4

        return(startingval - (ontoporbottom + onleftorright) + edgestosink);
    
    
    
# Iterates the given sandpile configuration on an mxn vertex
# rectangular graph multiple times (on each iteration firing 
# all vertices that can be fired), returning the resulting
# configuration. 
def lattice_iterate(m,n, sandpile_configuration, boundarytype=0, num_iterations=10, verbose_mode=True):
    for k in range(0,num_iterations):
        if (verbose_mode): print("starting iteration with configuration:")
        if (verbose_mode): print(np.reshape(sandpile_configuration,(m,n)))
        verticestotopple = []; 
        deg_matrix = lattice_degree_matrix(m,n,boundarytype);
        #print("degree matrix")
        #print(deg_matrix)
        adj_matrix = lattice_adjacency_matrix(m,n,boundarytype)
        #print(adj_matrix)
        #print("adjacency matrix")
        #print(adj_matrix)
        laplacian = np.subtract(deg_matrix, adj_matrix)
        for i in range(0, m*n):
            if (sandpile_configuration[i] >= num_neighbors(i,m,n,boundarytype)):
                verticestotopple.append(i);
        if (len(verticestotopple) == 0):
            if (verbose_mode): print("no vertices can be fired: the configuration is stable")
            return sandpile_configuration;
        for k in range(0, len(verticestotopple)):
            i = verticestotopple[k]
            sandpile_configuration = np.subtract(sandpile_configuration, laplacian[i]);
            if (verbose_mode): print("firing vertex number",i, "gives the configuration:");
            if (verbose_mode): print(np.reshape(sandpile_configuration,(m,n)));
        if (verbose_mode): print("iteration completed")
    return sandpile_configuration;
    
    
    

def nrows(M): return(len(M))    
def ncols(M): return(len(M[0]))
    
def Neighbors(M,i,j):
    c = ncols(M)
    r = nrows(M)
    rows = []
    cols = []
    if i>0:
        rows.append(i-1)
    if i<r-1:
        rows.append(i+1)
    if j>0:
        cols.append(j-1)
    if j<c-1:
        cols.append(j+1) 
    return [[a,j] for a in rows]+[[i,b] for b in cols]
        

def chipFire(M,i,j):  #### side effect on M!!!! 
    changed = False
    if M[i,j]>3:
        changed = True
        M[i,j] = M[i,j]-4
        for [a,b] in Neighbors(M,i,j):
            M[a,b] += 1
    return changed     

def blitzFire(M):
    changed = True
    while changed == True:
        changed = False
        for i in range(nrows(M)):
            for j in range(ncols(M)):
                if chipFire(M,i,j):
                    changed = True
    return(M)
 
#def squareShapeColor(i,j,size,color):
#    return polygon([(i,j),(i,j+size),(i+size,j+size),(i+size,j)],rgbcolor=color,fill=True)
        
def matrixPlot(M,colors):
    squares = []
    r = nrows(M)
    c = ncols(M)
    for i in range(r):
        for j in range(c):
            squares.append(squareShapeColor(j,r-i+1,1,colors[M[i,j]]))
    return sum(squares)

def SPconstant(c,m,n):
    M = constantMatrix(c,m,n)
    blitzFire(M)        
    return M


def SPidentityElem(m,n):
    M = constantMatrix(6,m,n)   ## 6 = 2*deg(v)-2
    blitzFire(M)
    M=constantMatrix(6,m,n)-M
    blitzFire(M)
    return M

import numpy as np
def constantMatrix(a,r,c):  ## a = value in every entry, r = rows, c=columns
    return np.full((r,c),a)

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from IPython.display import display
def f(gridsize,numit1,numit2):
    sigma = np.full(gridsize**2,6)
    stabsigmamatrix = blitzFire(np.reshape(sigma,(gridsize,gridsize)))
    sigmaminusstabsigma=np.full(gridsize**2,6) - np.reshape(stabsigmamatrix,(1,gridsize**2))[0]
    #print(sigmaminusstabsigma)
    display(np.reshape(lattice_iterate(gridsize,gridsize,np.full(gridsize**2,6),3,numit1,False),(gridsize,gridsize))
            ,np.reshape(lattice_iterate(gridsize,gridsize,sigmaminusstabsigma,3,numit2,False),(gridsize,gridsize)))
    
w = interactive(f, gridsize=widgets.IntSlider(min=1,max=300,step=1,value=5), 
                   numit1=widgets.IntSlider(min=0,max=300,step=1,value=0),
                   numit2=widgets.IntSlider(min=0,max=300,step=1,value=0))
display(w)

##interact(f,numit=widgets.IntSlider(min=0,max=300,step=1,value=10));